In [2]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%%bash 
pwd

/content/Template_Project


In [5]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)
from core.dl_framework.utils import get_specific_history
from core.dl_framework.utils import plot_history

In [ ]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor, EarlyStopping, Checkpoints
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(20)


In [17]:
class testA():
    def __init__(self, a): 
        self.a = a

if "testA" in globals():
    b = globals()["testA"](5)
vars(b)

{'a': 5}

In [6]:
from pathlib import Path
from datetime import datetime
ckp_path = "/content/gdrive/MyDrive/Marcel_Moczarski/Data_Science/Google_Services/Local_Drive_Storage/Conda_Envs/Template_Project/checkpoints"

In [5]:
# sample = next(iter(test_dl))
# imgs, targets = sample
from core.dl_framework.callbacks import load_checkpoint

In [8]:
best_hist = get_specific_history(Path(setup_config["c_c_ckp_path"]), "valid_loss", "parquet", "all")
plot_history(best_hist[-1])